In [ ]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import sqlite3



In [15]:
def contains_kanji(s: str) -> bool:
    return any('\u4E00' <= ch <= '\u9FFF' for ch in s)

def contains_katakana(s: str) -> bool:
    return any('\u30A0' <= ch <= '\u30FF' for ch in s)

In [16]:
df_dict = {i: pd.read_csv(f'https://raw.githubusercontent.com/jamsinclair/open-anki-jlpt-decks/refs/heads/main/src/n{i}.csv') for i in range(1,6)}
for n, df in df_dict.items():
    df.drop(df[~df.expression.apply(contains_kanji)].index, inplace=True)
    df.drop(df[df.reading.apply(contains_katakana)].index, inplace=True)
    df['vocab_set'] = f'jlpt{n}'

df_all = pd.concat(df_dict.values(), ignore_index=True)
df_all.drop_duplicates(subset=['expression'], keep=False, inplace=True)
df_all.drop(df_all[df_all.reading.str.contains(';', regex=False)].index, inplace=True)
df_all.drop(df_all[df_all.reading.str.contains(')', regex=False)].index, inplace=True)
df_all.drop(df_all[df_all.reading.str.contains('(', regex=False)].index, inplace=True)

df_all.drop(columns=['tags', 'guid'], inplace=True)
#Wrong reading in original
df_all.loc[~(df_all.expression.str.endswith('る')) & (df_all.reading.str.endswith('する')), 'reading'] = df_all.reading.str[0:-len('する')]
df_all.loc[(df_all.reading.str.endswith(' (する)')), 'reading'] = df_all.reading.str[0:-len(' (する)')]

In [17]:
with sqlite3.connect(r'..\..\android-app\app\src\main\assets\vocab.db') as conn:
    conn.execute("DROP TABLE IF EXISTS vocab;")

    conn.execute("""
    CREATE TABLE vocab (
        id INTEGER PRIMARY KEY NOT NULL,
        expression TEXT NOT NULL,
        reading TEXT NOT NULL,
        meaning TEXT NOT NULL,
        vocab_set TEXT NOT NULL
    );
    """)

    conn.commit()
    df_all.to_sql('vocab', conn, if_exists="append", index=False)
    conn.commit()


In [18]:
df_all[df_all.duplicated(['expression'])].sort_values(by=['expression'])

,expression,reading,meaning,vocab_set
0,現像,げんぞう,developing (film),jlpt1
1,原則,げんそく,"principle, general rule",jlpt1
2,見地,けんち,point of view,jlpt1
3,現地,げんち,"actual place, local",jlpt1
4,限定,げんてい,"limit, restriction",jlpt1
...,...,...,...,...
6920,分かる,わかる,to understand,jlpt5
6921,忘れる,わすれる,to forget,jlpt5
6924,渡す,わたす,to hand (something) over (v.t.); to get across,jlpt5
6925,渡る,わたる,"to cross over, to go across",jlpt5
